Background:
A new strain of ransomware has managed to penetrate several critical government networks and NSA has been called upon to assist in remediating the infection to prevent massive data losses. For each infected machine, an encrypted copy of the key needed to decrypt the ransomed files has been stored in a smart contract on the Ethereum blockchain* and is set to only be unlocked upon receipt of the ransom payment. Your mission is to ultimately (1) find a way to unlock the ransomware without giving in to the attacker’s demands and (2) figure out a way to recover all of the funds already paid by other victims. Are YOU up to the challenge?


* - for the purposes of this challenge, a private blockchain has been created with no real monetary value associated with the Ether. See http://www.ethdocs.org/en/latest/ for more information about Ethereum.



# Task 0
We have acquired a packet capture (PCAP) file showing network traffic between a newly infected victim computer and the attacker's listening post (LP). The LP is simply a server that is listening for incoming connections on a certain port. To get started, analyze the network capture and submit the IP address of the attacker's LP.


Downloads:
Task0/register.pcap

Thoughts behind answer:
Figure out how to open pcap file. Either wireshark or scapy would do the job. We see two ips 172.26.175.126 10.108.159.179. Since 172.26.175.126 received the most traffic. I assume this is the attacker. 

In [14]:
import  scapy.all  as s
data_file = "./Task0/register.pcap"
stats = s.rdpcap(data_file).res
total_receiver = {}
for r in stats:
    if r.payload.dst not in total_receiver:
        total_receiver[r.payload.dst] = 0
    total_receiver[r.payload.dst] =  total_receiver[r.payload.dst] + r.payload.len
    
print(total_receiver)
total_sender = {}

for r in stats:
    if r.payload.src not in total_sender:
        total_sender[r.payload.src] = 0
    total_sender[r.payload.src] =  total_sender[r.payload.src] + r.payload.len
print(total_sender)

{'10.108.159.179': 1856, '172.26.175.126': 2728}
{'10.108.159.179': 2728, '172.26.175.126': 1856}


# Task 2 
Whenever the ransomware infects a new victim, it calls out to the attacker LP with (1) a unique victim identifier, (2) encrypted copy of the key used to ransom victim files, and (3) a one-time passcode (OTP) used for authentication. The LP, in turn, connects to an Ethereum client and deploys a Ransom "smart" contract on the Ethereum blockchain that authenticates with other parts of the attacker's infrastructure. After authentication, the LP returns back to the ransomware (running on victim computer) information describing where the victim should send funds. Ransom payment is made to an Escrow contract, so the LP responds with the Escrow contract's blockchain address, a 20-byte value derived from a private key that corresponds to a unique account/contract on the blockchain. Ransom and Escrow smart contracts and attacker infrastructure will be explained more fully in subsequent tasks.


The ransomware removes itself from victim computers after completing infection and registration processes. Therefore, a complete ransomware executable hasn't yet been found. The only binaries it leaves behind are two shared libraries - libclient_comms.so and libclient_crypt.so. It's unclear if leaving these binaries on disk is an oversight or intentional. Your task is to analyze the ransomware client-registration communications protocol and submit the information requested below. To assist you, we have provided the two above-mentioned shared library binaries and a packet capture (pcap) file showing registration-related network traffic between victim computer and attacker LP. The library binaries were taken from the same victim computer shown in the pcap.


Please submit the following:

Victim Identifier
Encrypted Ransom Key
One-Time Passcode (OTP) used to authenticate the client to the ransomware LP
Escrow contract address

Downloads:

Network Traffic PCAP (register.pcap)
Malware communications library (libclient_comms.so)
Malware cryptography library (libclient_crypt.so)
Ransom Note (ransom_note.txt)

In [15]:
! objdump -D ./Task1/libclient_comms.so


./Task1/libclient_comms.so:	file format ELF64-x86-64

Disassembly of section .note.gnu.build-id:
.note.gnu.build-id:
     1c8:	04 00 	addb	$0, %al
     1ca:	00 00 	addb	%al, (%rax)
     1cc:	14 00 	adcb	$0, %al
     1ce:	00 00 	addb	%al, (%rax)
     1d0:	03 00 	addl	(%rax), %eax
     1d2:	00 00 	addb	%al, (%rax)
     1d4:	47 4e  <unknown>
     1d6:	55 	pushq	%rbp
     1d7:	00 31 	addb	%dh, (%rcx)
     1d9:	56 	pushq	%rsi
     1da:	1b 23 	sbbl	(%rbx), %esp
     1dc:	ac 	lodsb	(%rsi), %al
     1dd:	22 fb 	andb	%bl, %bh
     1df:	0c 33 	orb	$51, %al
     1e1:	a8 b8 	testb	$-72, %al
     1e3:	1d 07 6d 67 05 	sbbl	$90664199, %eax
     1e8:	42 06  <unknown>
     1ea:	15  <unknown>
     1eb:	f8 	clc
Disassembly of section .gnu.hash:
.gnu.hash:
     1f0:	03 00 	addl	(%rax), %eax
     1f2:	00 00 	addb	%al, (%rax)
     1f4:	15 00 00 00 01 	adcl	$16777216, %eax
     1f9:	00 00 	addb	%al, (%rax)
     1fb:	00 06 	addb	%al, (%rsi)
     1fd:	00 00 	addb	%al, (%rax)
     1ff:	00 88 c0 22 09 00 	addb	

nopw	(%rax,%rax)
     ba0:	5b 	popq	%rbx
     ba1:	31 c0 	xorl	%eax, %eax
     ba3:	5d 	popq	%rbp
     ba4:	41 5c 	popq	%r12
     ba6:	c3 	retq
     ba7:	66 0f 1f 84 00 00 00 00 00 	nopw	(%rax,%rax)

start_client:
     bb0:	41 57 	pushq	%r15
     bb2:	41 56 	pushq	%r14
     bb4:	48 89 fa 	movq	%rdi, %rdx
     bb7:	41 55 	pushq	%r13
     bb9:	41 54 	pushq	%r12
     bbb:	b9 f2 00 00 00 	movl	$242, %ecx
     bc0:	55 	pushq	%rbp
     bc1:	53 	pushq	%rbx
     bc2:	89 f5 	movl	%esi, %ebp
     bc4:	be 02 00 00 00 	movl	$2, %esi
     bc9:	48 81 ec d8 0e 00 00 	subq	$3800, %rsp
     bd0:	48 8d 5c 24 60 	leaq	96(%rsp), %rbx
     bd5:	48 c7 44 24 50 00 00 00 00 	movq	$0, 80(%rsp)
     bde:	48 c7 44 24 58 00 00 00 00 	movq	$0, 88(%rsp)
     be7:	64 48 8b 04 25 28 00 00 00 	movq	%fs:40, %rax
     bf0:	48 89 84 24 c8 0e 00 00 	movq	%rax, 3784(%rsp)
     bf8:	31 c0 	xorl	%eax, %eax
     bfa:	48 89 df 	movq	%rbx, %rdi
     bfd:	66 89 74 24 50 	movw	%si, 80(%rsp)
     c02:	f3 48 ab 	rep		stosq	%rax, %e